In [ ]:
"""
DESCRIPTION:
    This sample demonstrates how to use agent operations with the Bing grounding tool from
    the Azure Agents service using a synchronous client.

USAGE:
    python sample_agents_bing_grounding.py

    Before running the sample:

    pip install azure-ai-agents azure-identity

    Set these environment variables with your own values:
    1) PROJECT_ENDPOINT - The Azure AI Project endpoint, as found in the Overview
                          page of your Azure AI Foundry portal.
    2) MODEL_DEPLOYMENT_NAME - The deployment name of the AI model, as found under the "Name" column in
       the "Models + endpoints" tab in your Azure AI Foundry project.
    3) AZURE_BING_CONNECTION_ID - The ID of the Bing connection, in the format of:
       /subscriptions/{subscription-id}/resourceGroups/{resource-group-name}/providers/Microsoft.MachineLearningServices/workspaces/{workspace-name}/connections/{connection-name}
"""

from json import load
import os
from azure.ai.agents import AgentsClient
from azure.ai.agents.models import MessageRole, BingGroundingTool
from azure.identity import DefaultAzureCredential

load_dotenv = True
if load_dotenv:
    from dotenv import load_dotenv
    load_dotenv()


In [ ]:
agents_client = AgentsClient(
    endpoint=os.getenv("PROJECT_ENDPOINT"),
    credential=DefaultAzureCredential(),
)

In [ ]:
# [START create_agent_with_bing_grounding_tool]
conn_id = os.getenv("AZURE_BING_CONNECTION_ID")

# Initialize agent bing tool and add the connection id
bing = BingGroundingTool(connection_id=conn_id)

In [ ]:
agent = agents_client.create_agent(
        model=os.getenv("MODEL_DEPLOYMENT_NAME"),
        name="my-agent",
        instructions="You are a helpful agent",
        tools=bing.definitions,
    )
    # [END create_agent_with_bing_grounding_tool]

print(f"Created agent, ID: {agent.id}")


In [ ]:
# Create thread for communication
thread = agents_client.threads.create()
print(f"Created thread, ID: {thread.id}")

In [ ]:
# Create message to thread
message = agents_client.messages.create(
    thread_id=thread.id,
    role=MessageRole.USER,
    content="How does wikipedia explain Euler's Identity?",
)
print(f"Created message, ID: {message.id}")


In [ ]:
# Create and process agent run in thread with tools
run = agents_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

In [ ]:
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import ListSortOrder
load_dotenv = True
if load_dotenv:
    from dotenv import load_dotenv
    load_dotenv()

project = AIProjectClient(
    credential=DefaultAzureCredential(),
    endpoint="https://aq-ai-foundry-sweden-central.services.ai.azure.com/api/projects/firstProject")

# agent = project.agents.get_agent("asst_N2x3Nw5zEW9KfOOmwpy1I1NJ")
agent = project.agents.create_agent(
    model=os.getenv("MODEL_DEPLOYMENT_NAME"),
    name="my-agent",
    instructions="You are a helpful agent",
    tools=[{
        "type": "bing_grounding",
        "connection_id": os.getenv("AZURE_BING_CONNECTION_ID")
    }]
)
print(f"Created agent, ID: {agent.id}")
# Create thread for communication
thread = project.agents.threads.create(
    agent_id=agent.id,
    role="user",
    content="How does wikipedia explain Euler's Identity?"
)

print(f"Created thread, ID: {thread.id}")
# Create message to thread

message = project.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content="How does wikipedia explain Euler's Identity?"
)

print(f"Created message, ID: {message.id}")
# Create and process agent run in thread with tools

run = project.agents.runs.create_and_process(
    thread_id=thread.id,
    agent_id=agent.id)

if run.status == "failed":
    print(f"Run failed: {run.last_error}")
else:
    messages = project.agents.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)

    for message in messages:
        if message.text_messages:
            print(f"{message.role}: {message.text_messages[-1].text.value}")

In [5]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import BingGroundingTool
load_dotenv = True
if load_dotenv:
    from dotenv import load_dotenv
    load_dotenv()

# Create an Azure AI Client from an endpoint, copied from your Azure AI Foundry project.
# You need to login to Azure subscription via Azure CLI and set the environment variables
project_endpoint = os.getenv("PROJECT_ENDPOINT")  # Ensure the PROJECT_ENDPOINT environment variable is set

# Create an AIProjectClient instance
project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),  # Use Azure Default Credential for authentication
)

In [6]:
conn_id = os.environ["BING_CONNECTION_NAME"]  # Ensure the BING_CONNECTION_NAME environment variable is set

# Initialize the Bing Grounding tool
bing = BingGroundingTool(connection_id=conn_id)


agent = project_client.agents.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],  # Model deployment name
    name="my-agent",  # Name of the agent
    instructions="You are a helpful agent",  # Instructions for the agent
    tools=bing.definitions,  # Attach the Bing Grounding tool
)
print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_eZUMPIOpKLcWcLhgqPQfNTjS


In [7]:
# Create a thread for communication
thread = project_client.agents.threads.create()
print(f"Created thread, ID: {thread.id}")

# Add a message to the thread
message = project_client.agents.messages.create(
    thread_id=thread.id,
    role="user",  # Role of the message sender
    content="What is the weather in Seattle today?",  # Message content
)
print(f"Created message, ID: {message.id}")

Created thread, ID: thread_TLPlsLvJaDTFHGGObGn6HWqa
Created message, ID: msg_eA59jxX8kmtUw5wCckZ2YIMW


In [8]:
# Create and process an agent run
run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

# Check if the run failed
if run.status == "failed":
    print(f"Run failed: {run.last_error}")

# Fetch and log all messages
messages = project_client.agents.messages.list(thread_id=thread.id)
for message in messages:
    print(f"Role: {message.role}, Content: {message.content}")

# Delete the agent when done
project_client.agents.delete_agent(agent.id)
print("Deleted agent")

Run finished with status: RunStatus.FAILED
Run failed: {'code': 'tool_user_error', 'message': "Error: invalid_tool_input; The specified connection ID 'aqbinggrounding001' was not found in the project or account connections. Please verify that the connection id in tool input is correct and exists in the project or account."}
Role: MessageRole.USER, Content: [{'type': 'text', 'text': {'value': 'What is the weather in Seattle today?', 'annotations': []}}]
Deleted agent
